In [103]:
import networkx as nx
import re

In [104]:
G = nx.Graph()
with open('16.txt') as f:
    data = f.read().splitlines()
    
for line in data:
    line = line.replace('valves', 'valve')
    valve_name = line[6:8]
    left, right = line.split('; ')
    rest, flow_rate = left.split('=')
    flow_rate = int(flow_rate)
    destinations = right.split('valve ')
    if ',' in destinations[1]:
        destinations = destinations[1].split(', ')
    else:
        destinations = [destinations[1]]
    
    # NX
    G.add_node(valve_name, flow_rate=flow_rate, open=False)
    for dest in destinations:
        G.add_edge(valve_name, dest, weight=1)
    

In [105]:
from functools import lru_cache
import itertools

@lru_cache
def get_shortest_path(G, start, end):
    return nx.shortest_path(G, start, end)

In [106]:
class PressureFlow:
    def __init__(self, verbose=False):
        self.flow_rate = 0
        self.released_pressure = 0
        self.passed_minutes = 0
        self.path = []
    def increase_flow(self, amount):
        self.timestep()
        self.flow_rate += amount
    def timestep(self):
        self.released_pressure += self.flow_rate
        self.passed_minutes += 1
    def copy(self):
        pf = PressureFlow()
        pf.path = self.path.copy()
        pf.flow_rate = self.flow_rate
        pf.released_pressure = self.released_pressure
        pf.passed_minutes = self.passed_minutes
        return pf

In [107]:
max_minutes = 26

from tqdm import tqdm
from math import factorial
import copy


def move_and_maybe_open(G, flow: PressureFlow, current_node, node):
    shortest_path = get_shortest_path(G, current_node, node)
    for _ in range(len(shortest_path)-1):
        flow.timestep()
        if flow.passed_minutes >= max_minutes:
            return G, flow
    if not flow.passed_minutes >= max_minutes:
        open_node(G, flow, node)
    return G, flow

def open_node(G, flow: PressureFlow, node):
    # Open node if smart
    if (G.nodes[node]['flow_rate'] > 0) and not G.nodes[node]['open']:
        flow.increase_flow(G.nodes[node]["flow_rate"])
        G.nodes[node]['open'] = True
        
def calculate_potential(G, flow: PressureFlow, current_node, node):
    # Calculate potential
    if G.nodes[node]['open']:
        return 0
    shortest_path = get_shortest_path(G, current_node, node)
    potential_flow = (max_minutes - flow.passed_minutes - (len(shortest_path) - 1) - 1) * G.nodes[node]['flow_rate']
    return potential_flow

def get_best_opening_combination(G, flow, starting_node, depth=0):
    global best_flows
    possible_nodes = [x for x in G.nodes if calculate_potential(G, flow, starting_node, x) > 0]
    max_flow = None
    if len(possible_nodes) == 0 or flow.passed_minutes >= max_minutes:
        while flow.passed_minutes < max_minutes:
            flow.timestep()
        max_flow = flow
        best_flows.append(max_flow)
    else:
        if not max_flow:
            possible_next_nodes = []
            for node in possible_nodes:
                flow_ = flow.copy()
                flow_.path.append(node)
                G_, flow_ = move_and_maybe_open(G.copy(), flow_, starting_node, node)
                possible_next_nodes.append(get_best_opening_combination(G_, flow_, node, depth+1))
            max_flow = max([x.released_pressure for x in possible_next_nodes])
            max_flow = [x for x in possible_next_nodes if x.released_pressure == max_flow][0]
            best_flows.append(max_flow)
    return max_flow


In [108]:
flow = PressureFlow()
best_flows = []
max_flow = get_best_opening_combination(G.copy(), flow, 'AA')

In [109]:
len(best_flows)

42004

In [110]:
max_flow = max([x.released_pressure for x in best_flows])
best_flow = [x for x in best_flows if x.released_pressure ==  max_flow ][0]

In [111]:
print(f'Max flow: {max_flow}. Path: {best_flow.path}')

Max flow: 1669. Path: ['KQ', 'RF', 'AZ', 'VI', 'IM', 'IY', 'AQ']


In [112]:
a = set([1,2,3])
b = set([4,5])

In [113]:
if a&b:
    print('yes')

In [114]:
# Find two paths which have completely different paths

max_flow = 0
best_combination_flows = None
for flow in tqdm(best_flows):
    for flow_ in best_flows:
        if not set(flow.path) & set(flow_.path):
            flow_value = flow.released_pressure + flow_.released_pressure
            if flow_value > max_flow:
                max_flow = flow_value
                best_combination_flows = (flow, flow_)
                print(f'Flow 1: {flow.released_pressure} - {flow.path}')
                print(f'Flow 2: {flow_.released_pressure} - {flow_.path}')

  0%|          | 0/42004 [00:00<?, ?it/s]

Flow 1: 546 - ['LS', 'IY', 'VI', 'AZ', 'HA']
Flow 2: 529 - ['YC', 'QK', 'IM', 'AQ']
Flow 1: 546 - ['LS', 'IY', 'VI', 'AZ', 'HA']
Flow 2: 562 - ['YC', 'QK', 'KZ', 'IM', 'AQ']
Flow 1: 546 - ['LS', 'IY', 'VI', 'AZ', 'HA']
Flow 2: 578 - ['YC', 'QK', 'KZ', 'AQ', 'IM']
Flow 1: 546 - ['LS', 'IY', 'VI', 'AZ', 'HA']
Flow 2: 648 - ['YC', 'QK', 'KZ', 'HC', 'ZJ']
Flow 1: 546 - ['LS', 'IY', 'VI', 'AZ', 'HA']
Flow 2: 711 - ['YC', 'QK', 'KZ', 'HC', 'AI']
Flow 1: 546 - ['LS', 'IY', 'VI', 'AZ', 'HA']
Flow 2: 773 - ['YC', 'QK', 'KZ', 'RF', 'KQ', 'ZJ']
Flow 1: 546 - ['LS', 'IY', 'VI', 'AZ', 'HA']
Flow 2: 886 - ['YC', 'QK', 'HC', 'AI', 'KZ']
Flow 1: 546 - ['LS', 'IY', 'VI', 'AZ', 'HA']
Flow 2: 893 - ['YC', 'QK', 'HC', 'AI', 'ZJ']
Flow 1: 546 - ['LS', 'IY', 'VI', 'AZ', 'HA']
Flow 2: 898 - ['YC', 'QK', 'AI', 'HC', 'ZJ']
Flow 1: 546 - ['LS', 'IY', 'VI', 'AZ', 'HA']
Flow 2: 904 - ['YC', 'HC', 'QK', 'KZ', 'KQ', 'RF']
Flow 1: 546 - ['LS', 'IY', 'VI', 'AZ', 'HA']
Flow 2: 943 - ['YC', 'HC', 'QK', 'KZ', 'RF', 'KQ'

  0%|          | 4/42004 [00:00<20:17, 34.50it/s]

Flow 1: 498 - ['LS', 'IY', 'VI', 'IM']
Flow 2: 1311 - ['KQ', 'RF', 'AZ', 'AQ', 'KZ']


  0%|          | 25/42004 [00:00<14:47, 47.29it/s]

...

 29%|██▊       | 12038/42004 [04:02<10:12, 48.89it/s]

Flow 1: 1135 - ['QK', 'YC', 'HC', 'AI', 'LS', 'KZ']
Flow 2: 1612 - ['KQ', 'RF', 'AZ', 'IY', 'IM', 'AQ', 'HA']
Flow 1: 1135 - ['QK', 'YC', 'HC', 'AI', 'LS', 'KZ']
Flow 2: 1636 - ['KQ', 'RF', 'AZ', 'IY', 'AQ', 'IM', 'VI']
Flow 1: 1135 - ['QK', 'YC', 'HC', 'AI', 'LS', 'KZ']
Flow 2: 1650 - ['KQ', 'RF', 'AZ', 'VI', 'IY', 'AQ', 'IM']
Flow 1: 1135 - ['QK', 'YC', 'HC', 'AI', 'LS', 'KZ']
Flow 2: 1669 - ['KQ', 'RF', 'AZ', 'VI', 'IM', 'IY', 'AQ']
Flow 1: 1149 - ['QK', 'YC', 'HC', 'AI', 'KZ', 'LS']
Flow 2: 1669 - ['KQ', 'RF', 'AZ', 'VI', 'IM', 'IY', 'AQ']
Flow 1: 1160 - ['QK', 'YC', 'HC', 'AI', 'ZJ', 'LS']
Flow 2: 1669 - ['KQ', 'RF', 'AZ', 'VI', 'IM', 'IY', 'AQ']
Flow 1: 1164 - ['QK', 'YC', 'HC', 'AI', 'ZJ', 'KZ']
Flow 2: 1669 - ['KQ', 'RF', 'AZ', 'VI', 'IM', 'IY', 'AQ']


 30%|██▉       | 12488/42004 [04:11<10:17, 47.81it/s]

Flow 1: 1165 - ['QK', 'YC', 'AI', 'HC', 'ZJ', 'LS']
Flow 2: 1669 - ['KQ', 'RF', 'AZ', 'VI', 'IM', 'IY', 'AQ']
Flow 1: 1169 - ['QK', 'YC', 'AI', 'HC', 'ZJ', 'KZ']
Flow 2: 1669 - ['KQ', 'RF', 'AZ', 'VI', 'IM', 'IY', 'AQ']


100%|██████████| 42004/42004 [14:14<00:00, 49.17it/s]


In [115]:
best_combination_flows[0].path

['QK', 'YC', 'AI', 'HC', 'ZJ', 'KZ']

In [118]:
best_combination_flows[0].released_pressure

1169

In [116]:
best_combination_flows[1].path

['KQ', 'RF', 'AZ', 'VI', 'IM', 'IY', 'AQ']

In [117]:
best_combination_flows[1].released_pressure

1669

In [119]:
1169 + 1669

2838

In [226]:
## Part 2
max_minutes = 26

from tqdm import tqdm
from math import factorial
import copy

class Flow:
    def __init__(self, verbose=False):
        self.flow_rate = 0
        self.released_pressure = 0
        self.passed_minutes = 0
        self.path = []
        
    def increase_flow(self, G, node):
        self.timestep()
        self.flow_rate += G.nodes[node]["flow_rate"]
        self.path.append(node)
        
    def timestep(self):
        self.released_pressure += self.flow_rate
        self.passed_minutes += 1
        
    def copy(self):
        pf = Flow()
        pf.flow_rate = self.flow_rate
        pf.released_pressure = self.released_pressure
        pf.passed_minutes = self.passed_minutes
        return pf


    
def move_and_maybe_open(G, flow: PressureFlow, current_node, node):
    shortest_path = get_shortest_path(G, current_node, node)
    for _ in range(len(shortest_path)-1):
        flow.timestep()
        if flow.passed_minutes > max_minutes:
            return G, flow
    if flow.passed_minutes < max_minutes:
        open_node(G, flow, node)
    return G, flow

def open_node(G, flow: PressureFlow, node):
    # Open node if smart
    if (G.nodes[node]['flow_rate'] > 0) and not G.nodes[node]['open']:
        flow.increase_flow(G, node)
        G.nodes[node]['open'] = True
        
def calculate_potential(G, flow: PressureFlow, current_node, node):
    # Calculate potential
    if G.nodes[node]['open']:
        return 0
    shortest_path = get_shortest_path(G, current_node, node)
    potential_flow = (max_minutes - flow.passed_minutes - (len(shortest_path) - 1) - 1) * G.nodes[node]['flow_rate']
    return potential_flow

def get_best_opening_combination_2(G, flow_elf, flow_elephant, current_node_elf, current_node_elephant):
    possible_nodes_elf = [x for x in G.nodes if calculate_potential(G, flow_elf, current_node_elf, x) > 0]
    possible_nodes_elephant = [x for x in G.nodes if calculate_potential(G, flow_elephant, current_node_elephant, x) > 0]
    
    if len(possible_nodes_elf) == 0 and len(possible_nodes_elephant) == 0:
        while flow_elf.passed_minutes < max_minutes:
            flow_elf.timestep()
        while flow_elephant.passed_minutes < max_minutes:
            flow_elephant.timestep()
        return flow_elephant.released_pressure + flow_elf.released_pressure
    
    if len(possible_nodes_elephant) == 0:
        while flow_elephant.passed_minutes < max_minutes:
            flow_elephant.timestep()
        return flow_elephant.released_pressure + get_best_opening_combination(
            G.copy(), flow_elf, current_node_elf)
            
    if len(possible_nodes_elf) == 0:
        while flow_elf.passed_minutes < max_minutes:
            flow_elf.timestep()
        return flow_elf.released_pressure + get_best_opening_combination(
            G.copy(), flow_elephant, current_node_elephant)
        
    elif flow_elf.passed_minutes == max_minutes and flow_elephant.passed_minutes == max_minutes:
        return flow_elephant.released_pressure + flow_elf.released_pressure
    
    else:
        possible_next_nodes = []
        for node_elf in possible_nodes_elf:
            for node_elephant in possible_nodes_elephant:
                if node_elf == node_elephant and (len(possible_nodes_elf) > 1 or len(possible_nodes_elephant) > 1):
                    continue
                elif node_elf == node_elephant and len(possible_nodes_elf) == 1 and len(possible_nodes_elephant) == 1:
                    potential_elephant = calculate_potential(G, flow_elephant, current_node_elephant, node_elephant)
                    potential_elf = calculate_potential(G, flow_elf, current_node_elf, node_elf)
                    if potential_elephant > potential_elf:
                        while flow_elf.passed_minutes < max_minutes:
                            flow_elf.timestep()
                        return flow_elf.released_pressure + get_best_opening_combination(
                            G.copy(), flow_elephant, node_elephant)
                    elif potential_elephant <= potential_elf: # Smaller than because potential could be equal.
                        while flow_elephant.passed_minutes < max_minutes:
                            flow_elephant.timestep()
                        return flow_elephant.released_pressure + get_best_opening_combination(
                            G.copy(), flow_elf, node_elf)
                    else:
                        raise ValueError("Something went wrong")
                        
                else:
                    G_elf_, flow_elf_ = move_and_maybe_open(G.copy(), flow.copy(), current_node_elf, node_elf)
                    G_eleph_, flow_elephant_ = move_and_maybe_open(G.copy(), flow.copy(), current_node_elephant, node_elephant)

                    best_flow_elf = get_best_opening_combination_2(G_elf_.copy(), flow_elf_.copy(), flow_elephant_.copy(), current_node_elf, node_elf)
                    best_flow_ele = get_best_opening_combination_2(G_eleph_.copy(), flow_elf_.copy(), flow_elephant_.copy(), current_node_elephant, node_elephant)
                    possible_next_nodes.append(best_flow_elf + best_flow_ele)
        return max(possible_next_nodes) 


In [227]:
flow = Flow()

get_best_opening_combination_2(G.copy(), flow.copy(), flow.copy(), 'AA', 'AA')

KeyboardInterrupt: 